In [82]:
import re
import json
import numpy as np
import pandas as pd
import jieba
from zhconv import convert
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

In [83]:
# 读入20220821所有数据
all_data = pd.read_csv('../data/all_sample_20220821_spark.csv').drop(['Unnamed: 0'], axis=1)

/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_38964/3330071591.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.read_csv('../data/all_sample_20220821_spark.csv').drop(['Unnamed: 0'], axis=1)


In [84]:
# 显示所有的列
all_data.columns

Index(['cv_id', 'jd_id', 'jd_code', 'title', 'deadline', 'category_id',
       'category_name', 'company_id', 'company_name', 'recruit_number',
       'work_age', 'min_annual_salary', 'max_annual_salary',
       'min_month_salary', 'max_month_salary', 'pay_months', 'created_by_name',
       'created_by', 'created_at', 'updated_by', 'updated_at', 'tenant_id',
       'guarantee_period', 'status', 'recommended_rcn', 'tags', 'workplaces',
       'description', 'school', 'degree', 'post_level_id', 'requirement', 'ds',
       '_id', 'additionInfo', 'age', 'certificates', 'code', 'competenceScore',
       'completeness', 'countryCode', 'createdAt', 'createdBy',
       'currentCompany', 'currentPosition', 'currentSalary', 'dateOfBirth',
       'degree.1', 'desiredIndustry', 'desiredPosition', 'desiredSalary',
       'dutyTime', 'expectLocation', 'gender', 'importType', 'industry',
       'interviewTime', 'jobIntentStatus', 'languageSkills', 'languages',
       'location', 'maintainerId', 'majo

In [85]:
# 注意一下，这里的列名和sample pipeline中的列名有一点大小写区别，问题不大
cv_columns = ['cv_id', 'currentPosition', 'desiredPosition', 'industry', 'desiredIndustry', 'majorName', 'skills', 'eduTracks', 'jobTracks', 'projectTracks']
jd_columns = ['jd_id', 'title', 'category_name', 'tags', 'description', 'requirement']

In [86]:
jd_data = all_data[jd_columns]
jd_data

,jd_id,title,category_name,tags,description,requirement
0,2c9207157bcbb76b017bcde2b46d04cd,市场营销负责人,市场营销,"[""营销"",""管理"",""销售"",""战略规划"",""管理工作""]",1、岗位职责:2、主导制定公司营销战略规划并参与公司整体战略规划制定。3、全面管理公司的市场...,1、电商2、线下营销
1,2c9207157bcbb76b017bf7f2bbbb3153,会员管理经理,用户运营,"[""会员运营"",""会员管理"",""管理"",""营销"",""社群运营""]",1、岗位职责：2、1. 负责“九木杂物社”品牌全渠道会员体系的管理，规划品牌会员运营方向，包...,1、有会员管理、会员活动经验
2,2c9207157bf94d99017c108f769405e0,产品市场经理,产品经理,"[""EXCEL"",""OFFICE"",""PPT"",""UI""]",1、•Support regional strategic planning and mar...,1、find jd
3,2c9207157d2f0ec5017d5a9265ba33d9,海外区域运营经理/高级经理,运营,"[""管理"",""运营"",""团队管理"",""电商"",""游戏""]",1、负责短视频业务的运营策略制定，配合完成公司的营收业务指标；2、主导团队的业务方向，包括但...,1、本科及以上学历优先，3年以上社交、直播、游戏、电商或相关行业运营工作经验；2、英语听说读...
4,2c9207157d631abb017d6eed1c79139f,高级项目经理,高级管理,"[""项目管理"",""推广"",""归档"",""风险评估"",""交货""]",1、负责从产品建立到第一批生产交付的全过程项目管理;2、 负责实现产品要求、进度、成本、质量...,1、 本科及以上学历，计算机科学或相关专业。能用英语交流者优先;2、5年以上可穿戴相关产品项...
...,...,...,...,...,...,...
81291,8a69f6f8823a93b901823d891b761d51,技术经理,机械设备工程师,"[""材料"",""考核"",""维修"",""技术标准"",""不锈钢""]",职责描述：1、全面负责公司技术标准制、各岗位的工艺制定跟改善；2、重要设备参数收集管控包各棍...,任职要求：1、具备压延400系不锈钢材料技术相关工作经验，能对水扎四六棍扎异常原因分析解决、...
81292,8a69c468826ee9b40182831bedfc5e79,中央市场总监,市场总监,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...
81293,8a69f6f8823a93b9018247c30aca4e3d,中央市场总监,市场/营销,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...
81294,2c92071580dd3ab70180f18714e119c0,java开发工程师（不限方向）,Java,"[""java"",""微服务/性能优化"",""SQL"",""架构"",""需求分析""]",1、负责公司OA产品的功能规划、需求分析设计、技术实现和用户体验；2、负责所属模块的代码开发...,1、本科及以上学历，2年以上的Java开发及应用经验，计算机或者相关专业;2、良好的问题理解...


jd可以做3个向量
* title + category_name + tags
* description
* requirement

cv可以做4个向量：
* currentPosition + desiredPosition
* skills
* jobTracks
* projectTracks

向量1：title + category_name + tags

In [87]:
# step1：缺失值填充

jd_data['title'].fillna('', inplace=True)
jd_data['category_name'].fillna('', inplace=True)
jd_data['tags'].fillna('[]', inplace=True)

/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_38964/1876698629.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jd_data['title'].fillna('', inplace=True)
/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_38964/1876698629.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jd_data['category_name'].fillna('', inplace=True)
/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_38964/1876698629.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

In [88]:
# step2：定义新列，将title + category_name合并

col_name1 = 'title'
col_name2 = 'category_name'

def col_merge_str_fun(series):
    '''
    适用于文本列合并
    '''
    return series[col_name1] + ' ' + series[col_name2]

jd_data['title_category'] = jd_data.apply(col_merge_str_fun, axis=1)
jd_data

/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_38964/2397176896.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jd_data['title_category'] = jd_data.apply(col_merge_str_fun, axis=1)


,jd_id,title,category_name,tags,description,requirement,title_category
0,2c9207157bcbb76b017bcde2b46d04cd,市场营销负责人,市场营销,"[""营销"",""管理"",""销售"",""战略规划"",""管理工作""]",1、岗位职责:2、主导制定公司营销战略规划并参与公司整体战略规划制定。3、全面管理公司的市场...,1、电商2、线下营销,市场营销负责人 市场营销
1,2c9207157bcbb76b017bf7f2bbbb3153,会员管理经理,用户运营,"[""会员运营"",""会员管理"",""管理"",""营销"",""社群运营""]",1、岗位职责：2、1. 负责“九木杂物社”品牌全渠道会员体系的管理，规划品牌会员运营方向，包...,1、有会员管理、会员活动经验,会员管理经理 用户运营
2,2c9207157bf94d99017c108f769405e0,产品市场经理,产品经理,"[""EXCEL"",""OFFICE"",""PPT"",""UI""]",1、•Support regional strategic planning and mar...,1、find jd,产品市场经理 产品经理
3,2c9207157d2f0ec5017d5a9265ba33d9,海外区域运营经理/高级经理,运营,"[""管理"",""运营"",""团队管理"",""电商"",""游戏""]",1、负责短视频业务的运营策略制定，配合完成公司的营收业务指标；2、主导团队的业务方向，包括但...,1、本科及以上学历优先，3年以上社交、直播、游戏、电商或相关行业运营工作经验；2、英语听说读...,海外区域运营经理/高级经理 运营
4,2c9207157d631abb017d6eed1c79139f,高级项目经理,高级管理,"[""项目管理"",""推广"",""归档"",""风险评估"",""交货""]",1、负责从产品建立到第一批生产交付的全过程项目管理;2、 负责实现产品要求、进度、成本、质量...,1、 本科及以上学历，计算机科学或相关专业。能用英语交流者优先;2、5年以上可穿戴相关产品项...,高级项目经理 高级管理
...,...,...,...,...,...,...,...
81291,8a69f6f8823a93b901823d891b761d51,技术经理,机械设备工程师,"[""材料"",""考核"",""维修"",""技术标准"",""不锈钢""]",职责描述：1、全面负责公司技术标准制、各岗位的工艺制定跟改善；2、重要设备参数收集管控包各棍...,任职要求：1、具备压延400系不锈钢材料技术相关工作经验，能对水扎四六棍扎异常原因分析解决、...,技术经理 机械设备工程师
81292,8a69c468826ee9b40182831bedfc5e79,中央市场总监,市场总监,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场总监
81293,8a69f6f8823a93b9018247c30aca4e3d,中央市场总监,市场/营销,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场/营销
81294,2c92071580dd3ab70180f18714e119c0,java开发工程师（不限方向）,Java,"[""java"",""微服务/性能优化"",""SQL"",""架构"",""需求分析""]",1、负责公司OA产品的功能规划、需求分析设计、技术实现和用户体验；2、负责所属模块的代码开发...,1、本科及以上学历，2年以上的Java开发及应用经验，计算机或者相关专业;2、良好的问题理解...,java开发工程师（不限方向） Java


In [89]:
# step3：将新列title_category通过jieba分词处理成列表

col_name = 'title_category'

def col_jieba_fun(series):
    col = series[col_name]

    # 字符串变列表
    if col.startswith("[") and col.endswith("]"):
        col = json.loads(col)
    else:
        col = re.split(",|，|/| ", col)

    # 对于中文，进入jieba前不需要添加空格
    # 不过，如果是中英文混合，就必须空格了
    col_str = " ".join(col)
    col_list = jieba.lcut(col_str, cut_all=False)
    return col_list

jd_data['title_category_jieba'] = jd_data.apply(col_jieba_fun, axis=1)
jd_data


/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_38964/1189197695.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jd_data['title_category_jieba'] = jd_data.apply(col_jieba_fun, axis=1)


,jd_id,title,category_name,tags,description,requirement,title_category,title_category_jieba
0,2c9207157bcbb76b017bcde2b46d04cd,市场营销负责人,市场营销,"[""营销"",""管理"",""销售"",""战略规划"",""管理工作""]",1、岗位职责:2、主导制定公司营销战略规划并参与公司整体战略规划制定。3、全面管理公司的市场...,1、电商2、线下营销,市场营销负责人 市场营销,"[市场营销, 负责人, , 市场营销]"
1,2c9207157bcbb76b017bf7f2bbbb3153,会员管理经理,用户运营,"[""会员运营"",""会员管理"",""管理"",""营销"",""社群运营""]",1、岗位职责：2、1. 负责“九木杂物社”品牌全渠道会员体系的管理，规划品牌会员运营方向，包...,1、有会员管理、会员活动经验,会员管理经理 用户运营,"[会员, 管理, 经理, , 用户, 运营]"
2,2c9207157bf94d99017c108f769405e0,产品市场经理,产品经理,"[""EXCEL"",""OFFICE"",""PPT"",""UI""]",1、•Support regional strategic planning and mar...,1、find jd,产品市场经理 产品经理,"[产品, 市场, 经理, , 产品, 经理]"
3,2c9207157d2f0ec5017d5a9265ba33d9,海外区域运营经理/高级经理,运营,"[""管理"",""运营"",""团队管理"",""电商"",""游戏""]",1、负责短视频业务的运营策略制定，配合完成公司的营收业务指标；2、主导团队的业务方向，包括但...,1、本科及以上学历优先，3年以上社交、直播、游戏、电商或相关行业运营工作经验；2、英语听说读...,海外区域运营经理/高级经理 运营,"[海外, 区域, 运营, 经理, , 高级, 经理, , 运营]"
4,2c9207157d631abb017d6eed1c79139f,高级项目经理,高级管理,"[""项目管理"",""推广"",""归档"",""风险评估"",""交货""]",1、负责从产品建立到第一批生产交付的全过程项目管理;2、 负责实现产品要求、进度、成本、质量...,1、 本科及以上学历，计算机科学或相关专业。能用英语交流者优先;2、5年以上可穿戴相关产品项...,高级项目经理 高级管理,"[高级, 项目经理, , 高级, 管理]"
...,...,...,...,...,...,...,...,...
81291,8a69f6f8823a93b901823d891b761d51,技术经理,机械设备工程师,"[""材料"",""考核"",""维修"",""技术标准"",""不锈钢""]",职责描述：1、全面负责公司技术标准制、各岗位的工艺制定跟改善；2、重要设备参数收集管控包各棍...,任职要求：1、具备压延400系不锈钢材料技术相关工作经验，能对水扎四六棍扎异常原因分析解决、...,技术经理 机械设备工程师,"[技术, 经理, , 机械设备, 工程师]"
81292,8a69c468826ee9b40182831bedfc5e79,中央市场总监,市场总监,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场总监,"[中央, 市场总监, , 市场总监]"
81293,8a69f6f8823a93b9018247c30aca4e3d,中央市场总监,市场/营销,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场/营销,"[中央, 市场总监, , 市场, , 营销]"
81294,2c92071580dd3ab70180f18714e119c0,java开发工程师（不限方向）,Java,"[""java"",""微服务/性能优化"",""SQL"",""架构"",""需求分析""]",1、负责公司OA产品的功能规划、需求分析设计、技术实现和用户体验；2、负责所属模块的代码开发...,1、本科及以上学历，2年以上的Java开发及应用经验，计算机或者相关专业;2、良好的问题理解...,java开发工程师（不限方向） Java,"[java, 开发, 工程师, （, 不, 限, 方向, ）, , Java]"


In [90]:
# step4：将tags列通过jieba分词处理成列表

col_name = 'tags'

def col_jieba_fun(series):
    col = series[col_name]

    # 字符串变列表
    if col.startswith("[") and col.endswith("]"):
        col = json.loads(col)
    else:
        col = re.split(",|，|/| ", col)

    # 对于中文，进入jieba前不需要添加空格
    # 不过，如果是中英文混合，就必须空格了
    col_str = " ".join(col)
    col_list = jieba.lcut(col_str, cut_all=False)
    return col_list

jd_data['tags_jieba'] = jd_data.apply(col_jieba_fun, axis=1)
jd_data


/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_38964/4288899022.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jd_data['tags_jieba'] = jd_data.apply(col_jieba_fun, axis=1)


,jd_id,title,category_name,tags,description,requirement,title_category,title_category_jieba,tags_jieba
0,2c9207157bcbb76b017bcde2b46d04cd,市场营销负责人,市场营销,"[""营销"",""管理"",""销售"",""战略规划"",""管理工作""]",1、岗位职责:2、主导制定公司营销战略规划并参与公司整体战略规划制定。3、全面管理公司的市场...,1、电商2、线下营销,市场营销负责人 市场营销,"[市场营销, 负责人, , 市场营销]","[营销, , 管理, , 销售, , 战略规划, , 管理工作]"
1,2c9207157bcbb76b017bf7f2bbbb3153,会员管理经理,用户运营,"[""会员运营"",""会员管理"",""管理"",""营销"",""社群运营""]",1、岗位职责：2、1. 负责“九木杂物社”品牌全渠道会员体系的管理，规划品牌会员运营方向，包...,1、有会员管理、会员活动经验,会员管理经理 用户运营,"[会员, 管理, 经理, , 用户, 运营]","[会员, 运营, , 会员, 管理, , 管理, , 营销, , 社群, 运营]"
2,2c9207157bf94d99017c108f769405e0,产品市场经理,产品经理,"[""EXCEL"",""OFFICE"",""PPT"",""UI""]",1、•Support regional strategic planning and mar...,1、find jd,产品市场经理 产品经理,"[产品, 市场, 经理, , 产品, 经理]","[EXCEL, , OFFICE, , PPT, , UI]"
3,2c9207157d2f0ec5017d5a9265ba33d9,海外区域运营经理/高级经理,运营,"[""管理"",""运营"",""团队管理"",""电商"",""游戏""]",1、负责短视频业务的运营策略制定，配合完成公司的营收业务指标；2、主导团队的业务方向，包括但...,1、本科及以上学历优先，3年以上社交、直播、游戏、电商或相关行业运营工作经验；2、英语听说读...,海外区域运营经理/高级经理 运营,"[海外, 区域, 运营, 经理, , 高级, 经理, , 运营]","[管理, , 运营, , 团队, 管理, , 电商, , 游戏]"
4,2c9207157d631abb017d6eed1c79139f,高级项目经理,高级管理,"[""项目管理"",""推广"",""归档"",""风险评估"",""交货""]",1、负责从产品建立到第一批生产交付的全过程项目管理;2、 负责实现产品要求、进度、成本、质量...,1、 本科及以上学历，计算机科学或相关专业。能用英语交流者优先;2、5年以上可穿戴相关产品项...,高级项目经理 高级管理,"[高级, 项目经理, , 高级, 管理]","[项目管理, , 推广, , 归档, , 风险, 评估, , 交货]"
...,...,...,...,...,...,...,...,...,...
81291,8a69f6f8823a93b901823d891b761d51,技术经理,机械设备工程师,"[""材料"",""考核"",""维修"",""技术标准"",""不锈钢""]",职责描述：1、全面负责公司技术标准制、各岗位的工艺制定跟改善；2、重要设备参数收集管控包各棍...,任职要求：1、具备压延400系不锈钢材料技术相关工作经验，能对水扎四六棍扎异常原因分析解决、...,技术经理 机械设备工程师,"[技术, 经理, , 机械设备, 工程师]","[材料, , 考核, , 维修, , 技术标准, , 不锈钢]"
81292,8a69c468826ee9b40182831bedfc5e79,中央市场总监,市场总监,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场总监,"[中央, 市场总监, , 市场总监]","[市场推广, , 医疗器械, , 语言, , 材料, , 营销]"
81293,8a69f6f8823a93b9018247c30aca4e3d,中央市场总监,市场/营销,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场/营销,"[中央, 市场总监, , 市场, , 营销]","[市场推广, , 医疗器械, , 语言, , 材料, , 营销]"
81294,2c92071580dd3ab70180f18714e119c0,java开发工程师（不限方向）,Java,"[""java"",""微服务/性能优化"",""SQL"",""架构"",""需求分析""]",1、负责公司OA产品的功能规划、需求分析设计、技术实现和用户体验；2、负责所属模块的代码开发...,1、本科及以上学历，2年以上的Java开发及应用经验，计算机或者相关专业;2、良好的问题理解...,java开发工程师（不限方向） Java,"[java, 开发, 工程师, （, 不, 限, 方向, ）, , Java]","[java, , 微, 服务, /, 性能, 优化, , SQL, , 架构, , ..."


In [91]:
# step5：将title_category_jieba和tags_jieba两个列表合并

col_name1 = 'title_category_jieba'
col_name2 = 'tags_jieba'

def col_merge_list_fun(series):
    '''
    适用于列表列合并
    '''
    return series[col_name1] + series[col_name2]

jd_data['title_category_tags_jieba'] = jd_data.apply(col_merge_list_fun, axis=1)
jd_data


/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_38964/1961427277.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jd_data['title_category_tags_jieba'] = jd_data.apply(col_merge_list_fun, axis=1)


,jd_id,title,category_name,tags,description,requirement,title_category,title_category_jieba,tags_jieba,title_category_tags_jieba
0,2c9207157bcbb76b017bcde2b46d04cd,市场营销负责人,市场营销,"[""营销"",""管理"",""销售"",""战略规划"",""管理工作""]",1、岗位职责:2、主导制定公司营销战略规划并参与公司整体战略规划制定。3、全面管理公司的市场...,1、电商2、线下营销,市场营销负责人 市场营销,"[市场营销, 负责人, , 市场营销]","[营销, , 管理, , 销售, , 战略规划, , 管理工作]","[市场营销, 负责人, , 市场营销, 营销, , 管理, , 销售, , 战略规划..."
1,2c9207157bcbb76b017bf7f2bbbb3153,会员管理经理,用户运营,"[""会员运营"",""会员管理"",""管理"",""营销"",""社群运营""]",1、岗位职责：2、1. 负责“九木杂物社”品牌全渠道会员体系的管理，规划品牌会员运营方向，包...,1、有会员管理、会员活动经验,会员管理经理 用户运营,"[会员, 管理, 经理, , 用户, 运营]","[会员, 运营, , 会员, 管理, , 管理, , 营销, , 社群, 运营]","[会员, 管理, 经理, , 用户, 运营, 会员, 运营, , 会员, 管理, , ..."
2,2c9207157bf94d99017c108f769405e0,产品市场经理,产品经理,"[""EXCEL"",""OFFICE"",""PPT"",""UI""]",1、•Support regional strategic planning and mar...,1、find jd,产品市场经理 产品经理,"[产品, 市场, 经理, , 产品, 经理]","[EXCEL, , OFFICE, , PPT, , UI]","[产品, 市场, 经理, , 产品, 经理, EXCEL, , OFFICE, , P..."
3,2c9207157d2f0ec5017d5a9265ba33d9,海外区域运营经理/高级经理,运营,"[""管理"",""运营"",""团队管理"",""电商"",""游戏""]",1、负责短视频业务的运营策略制定，配合完成公司的营收业务指标；2、主导团队的业务方向，包括但...,1、本科及以上学历优先，3年以上社交、直播、游戏、电商或相关行业运营工作经验；2、英语听说读...,海外区域运营经理/高级经理 运营,"[海外, 区域, 运营, 经理, , 高级, 经理, , 运营]","[管理, , 运营, , 团队, 管理, , 电商, , 游戏]","[海外, 区域, 运营, 经理, , 高级, 经理, , 运营, 管理, , 运营, ..."
4,2c9207157d631abb017d6eed1c79139f,高级项目经理,高级管理,"[""项目管理"",""推广"",""归档"",""风险评估"",""交货""]",1、负责从产品建立到第一批生产交付的全过程项目管理;2、 负责实现产品要求、进度、成本、质量...,1、 本科及以上学历，计算机科学或相关专业。能用英语交流者优先;2、5年以上可穿戴相关产品项...,高级项目经理 高级管理,"[高级, 项目经理, , 高级, 管理]","[项目管理, , 推广, , 归档, , 风险, 评估, , 交货]","[高级, 项目经理, , 高级, 管理, 项目管理, , 推广, , 归档, , 风..."
...,...,...,...,...,...,...,...,...,...,...
81291,8a69f6f8823a93b901823d891b761d51,技术经理,机械设备工程师,"[""材料"",""考核"",""维修"",""技术标准"",""不锈钢""]",职责描述：1、全面负责公司技术标准制、各岗位的工艺制定跟改善；2、重要设备参数收集管控包各棍...,任职要求：1、具备压延400系不锈钢材料技术相关工作经验，能对水扎四六棍扎异常原因分析解决、...,技术经理 机械设备工程师,"[技术, 经理, , 机械设备, 工程师]","[材料, , 考核, , 维修, , 技术标准, , 不锈钢]","[技术, 经理, , 机械设备, 工程师, 材料, , 考核, , 维修, , 技术..."
81292,8a69c468826ee9b40182831bedfc5e79,中央市场总监,市场总监,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场总监,"[中央, 市场总监, , 市场总监]","[市场推广, , 医疗器械, , 语言, , 材料, , 营销]","[中央, 市场总监, , 市场总监, 市场推广, , 医疗器械, , 语言, , 材..."
81293,8a69f6f8823a93b9018247c30aca4e3d,中央市场总监,市场/营销,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场/营销,"[中央, 市场总监, , 市场, , 营销]","[市场推广, , 医疗器械, , 语言, , 材料, , 营销]","[中央, 市场总监, , 市场, , 营销, 市场推广, , 医疗器械, , 语言,..."
81294,2c92071580dd3ab70180f18714e119c0,java开发工程师（不限方向）,Java,"[""java"",""微服务/性能优化"",""SQL"",""架构"",""需求分析""]",1、负责公司OA产品的功能规划、需求分析设计、技术实现和用户体验；2、负责所属模块的代码开发...,1、本科及以上学历，2年以上的Java开发及应用经验，计算机或者相关专业;2、良好的问题理解...,java开发工程师（不限方向） Java,"[java, 开发, 工程师, （, 不, 限, 方向, ）, , Java]","[java, , 微, 服务, /, 性能, 优化, , SQL, , 架构, , ...","[java, 开发, 工程师, （, 不, 限, 方向, ）, , Java, java,..."


In [92]:
# step6：过滤title_category_tags_jieba, 得到以空格分割的字符串

col_name = 'title_category_tags_jieba'
def col_jieba_filter_fun(series):

    col_list_filter = []
    # 得到tag列表
    col_list = series[col_name]
    # print(tags_list[0])

    pun_masks_english = [",", ".", "/", "[", "]", "{", "}", "(", ")", ":", "*", "#", "!", " ", "\"", "\\"]
    pun_masks_chinese = ["，", "。", "、", "（", "）", "：", "！", "”", "“"]
    pun_masks = pun_masks_english + pun_masks_chinese

    # 过滤
    for tag in col_list:
        # 转中文简体
        tag = convert(tag, "zh-hans")
        # 转英文小写
        tag = tag.lower()

        # 过滤数字
        if tag.isdigit():
            continue
        
        # 过滤单个字符
        if len(tag) <= 1:
            continue
        
        # 过滤标点
        flag = 1
        for pun in pun_masks:
            if pun in tag:
                flag = 0
                break
        if flag == 1:
            col_list_filter.append(tag)
    return " ".join(col_list_filter)

jd_data['title_category_tags_jieba_filter'] = jd_data.apply(col_jieba_filter_fun, axis=1)
jd_data

/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_38964/1399396701.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jd_data['title_category_tags_jieba_filter'] = jd_data.apply(col_jieba_filter_fun, axis=1)


,jd_id,title,category_name,tags,description,requirement,title_category,title_category_jieba,tags_jieba,title_category_tags_jieba,title_category_tags_jieba_filter
0,2c9207157bcbb76b017bcde2b46d04cd,市场营销负责人,市场营销,"[""营销"",""管理"",""销售"",""战略规划"",""管理工作""]",1、岗位职责:2、主导制定公司营销战略规划并参与公司整体战略规划制定。3、全面管理公司的市场...,1、电商2、线下营销,市场营销负责人 市场营销,"[市场营销, 负责人, , 市场营销]","[营销, , 管理, , 销售, , 战略规划, , 管理工作]","[市场营销, 负责人, , 市场营销, 营销, , 管理, , 销售, , 战略规划...",市场营销 负责人 市场营销 营销 管理 销售 战略规划 管理工作
1,2c9207157bcbb76b017bf7f2bbbb3153,会员管理经理,用户运营,"[""会员运营"",""会员管理"",""管理"",""营销"",""社群运营""]",1、岗位职责：2、1. 负责“九木杂物社”品牌全渠道会员体系的管理，规划品牌会员运营方向，包...,1、有会员管理、会员活动经验,会员管理经理 用户运营,"[会员, 管理, 经理, , 用户, 运营]","[会员, 运营, , 会员, 管理, , 管理, , 营销, , 社群, 运营]","[会员, 管理, 经理, , 用户, 运营, 会员, 运营, , 会员, 管理, , ...",会员 管理 经理 用户 运营 会员 运营 会员 管理 管理 营销 社群 运营
2,2c9207157bf94d99017c108f769405e0,产品市场经理,产品经理,"[""EXCEL"",""OFFICE"",""PPT"",""UI""]",1、•Support regional strategic planning and mar...,1、find jd,产品市场经理 产品经理,"[产品, 市场, 经理, , 产品, 经理]","[EXCEL, , OFFICE, , PPT, , UI]","[产品, 市场, 经理, , 产品, 经理, EXCEL, , OFFICE, , P...",产品 市场 经理 产品 经理 excel office ppt ui
3,2c9207157d2f0ec5017d5a9265ba33d9,海外区域运营经理/高级经理,运营,"[""管理"",""运营"",""团队管理"",""电商"",""游戏""]",1、负责短视频业务的运营策略制定，配合完成公司的营收业务指标；2、主导团队的业务方向，包括但...,1、本科及以上学历优先，3年以上社交、直播、游戏、电商或相关行业运营工作经验；2、英语听说读...,海外区域运营经理/高级经理 运营,"[海外, 区域, 运营, 经理, , 高级, 经理, , 运营]","[管理, , 运营, , 团队, 管理, , 电商, , 游戏]","[海外, 区域, 运营, 经理, , 高级, 经理, , 运营, 管理, , 运营, ...",海外 区域 运营 经理 高级 经理 运营 管理 运营 团队 管理 电商 游戏
4,2c9207157d631abb017d6eed1c79139f,高级项目经理,高级管理,"[""项目管理"",""推广"",""归档"",""风险评估"",""交货""]",1、负责从产品建立到第一批生产交付的全过程项目管理;2、 负责实现产品要求、进度、成本、质量...,1、 本科及以上学历，计算机科学或相关专业。能用英语交流者优先;2、5年以上可穿戴相关产品项...,高级项目经理 高级管理,"[高级, 项目经理, , 高级, 管理]","[项目管理, , 推广, , 归档, , 风险, 评估, , 交货]","[高级, 项目经理, , 高级, 管理, 项目管理, , 推广, , 归档, , 风...",高级 项目经理 高级 管理 项目管理 推广 归档 风险 评估 交货
...,...,...,...,...,...,...,...,...,...,...,...
81291,8a69f6f8823a93b901823d891b761d51,技术经理,机械设备工程师,"[""材料"",""考核"",""维修"",""技术标准"",""不锈钢""]",职责描述：1、全面负责公司技术标准制、各岗位的工艺制定跟改善；2、重要设备参数收集管控包各棍...,任职要求：1、具备压延400系不锈钢材料技术相关工作经验，能对水扎四六棍扎异常原因分析解决、...,技术经理 机械设备工程师,"[技术, 经理, , 机械设备, 工程师]","[材料, , 考核, , 维修, , 技术标准, , 不锈钢]","[技术, 经理, , 机械设备, 工程师, 材料, , 考核, , 维修, , 技术...",技术 经理 机械设备 工程师 材料 考核 维修 技术标准 不锈钢
81292,8a69c468826ee9b40182831bedfc5e79,中央市场总监,市场总监,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场总监,"[中央, 市场总监, , 市场总监]","[市场推广, , 医疗器械, , 语言, , 材料, , 营销]","[中央, 市场总监, , 市场总监, 市场推广, , 医疗器械, , 语言, , 材...",中央 市场总监 市场总监 市场推广 医疗器械 语言 材料 营销
81293,8a69f6f8823a93b9018247c30aca4e3d,中央市场总监,市场/营销,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场/营销,"[中央, 市场总监, , 市场, , 营销]","[市场推广, , 医疗器械, , 语言, , 材料, , 营销]","[中央, 市场总监, , 市场, , 营销, 市场推广, , 医疗器械, , 语言,...",中央 市场总监 市场 营销 市场推广 医疗器械 语言 材料 营销
81294,2c92071580dd3ab70180f18714e119c0,java开发工程师（不限方向）,Java,"[""java"",""微服务/性能优化"",""SQL"",""架构"",""需求分析""]",1、负责公司OA产品的功能规划、需求分析设计、技术实现和用户体验；2、负责所属模块的代码开发...,1、本科及以上学历，2年以上的Java开发及应用经验，计算机或者相关专业;2、良好的问题理解...,java开发工程师（不限方向） Java,"[java, 开发, 工程师, （, 不, 限, 方向, ）, , Java]","[java, , 微, 服务, /, 性能, 优化, , SQL, , 架构, , ...","[java, 开发, 工程师, （, 不, 限, 方向, ）, , Java, java,...",java 开发 工程师 方向 java java 服务 性能 优化 sql 架构 需求 分析


In [93]:
# step7：将title_category_tags_jieba_filter转成tfidf向量

def get_tfidf(df, col_name):
    # col_name必须是以空格分割的字符串
    text = df[col_name]
    
    vectorizer = TfidfVectorizer()
    # 返回的是nparray
    vector = vectorizer.fit_transform(text)
    return pd.DataFrame(vector.toarray()), vectorizer

tfidf, vectorizer = get_tfidf(jd_data, 'title_category_tags_jieba_filter')
tfidf


,0,1,2,3,4,5,6,7,8,9,...,3597,3598,3599,3600,3601,3602,3603,3604,3605,3606
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81291,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81292,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81293,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [94]:
vectorizer.vocabulary_

{'市场营销': 1878,
 '负责人': 3210,
 '营销': 3076,
 '管理': 2826,
 '销售': 3454,
 '战略规划': 2035,
 '管理工作': 2832,
 '会员': 1136,
 '经理': 2904,
 '用户': 2633,
 '运营': 3336,
 '社群': 2753,
 '产品': 1053,
 '市场': 1872,
 'excel': 288,
 'office': 597,
 'ppt': 665,
 'ui': 856,
 '海外': 2505,
 '区域': 1421,
 '高级': 3594,
 '团队': 1598,
 '电商': 2647,
 '游戏': 2544,
 '项目经理': 3520,
 '项目管理': 3519,
 '推广': 2143,
 '归档': 1966,
 '风险': 3550,
 '评估': 3178,
 '交货': 1049,
 '阿里': 3473,
 '智能': 2321,
 '金融': 3422,
 '行业': 3101,
 '解决方案': 3134,
 '架构师': 2384,
 '公有': 1257,
 'it': 442,
 '技术支持': 2072,
 '数据库': 2227,
 'devops': 231,
 '架构': 2383,
 '售前': 1564,
 '软件产品': 3305,
 'os': 611,
 '沟通': 2459,
 '能力': 3023,
 '软件': 3304,
 '测试': 2493,
 '反馈': 1502,
 '通信': 3366,
 '专卖店': 948,
 '店长': 1931,
 '门店': 3464,
 '调配': 3206,
 '信息': 1187,
 '收集': 2187,
 '旅行': 2281,
 '攻略': 2191,
 '内容': 1279,
 '策划': 2809,
 '调度': 3200,
 'ip': 430,
 '声学': 1676,
 '工程师': 1855,
 '技术': 2067,
 '音频': 3515,
 'ws': 907,
 '审核': 1776,
 '耳机': 2996,
 '结构设计': 2916,
 '仿真': 1121,
 'dsp': 253,
 'cocos2d': 1

In [95]:
# step8：将tfidf降维到n维

def get_tfidf_pca(tfidf, n=20):
    pca = PCA(n_components=n)
    tfidf_pca = pca.fit_transform(tfidf)
    tfidf_pca = pd.DataFrame(tfidf_pca)
    return tfidf_pca

tfidf_pca = get_tfidf_pca(tfidf, 10)
tfidf_pca

,0,1,2,3,4,5,6,7,8,9
0,-0.042800,-0.158423,-0.004030,0.164219,-0.007701,0.003777,0.000735,-0.022271,-0.033408,-0.029596
1,-0.236844,0.085824,-0.230642,-0.025113,-0.002692,0.011309,0.020816,-0.037760,0.001377,-0.037811
2,-0.190874,0.194129,0.343116,0.055046,0.033955,0.041350,-0.054385,0.004474,-0.048003,0.073818
3,-0.372099,0.191217,-0.314979,-0.012926,-0.015009,0.001544,0.020620,0.000789,-0.028087,0.109588
4,0.005622,-0.057164,0.001190,-0.038642,0.003226,0.000414,-0.038819,-0.022147,-0.066715,-0.073734
...,...,...,...,...,...,...,...,...,...,...
81291,0.034486,-0.046804,0.050006,-0.045027,-0.024668,-0.013060,-0.034595,-0.014761,0.001341,-0.026375
81292,0.015164,-0.080300,0.009701,-0.010164,-0.032643,-0.000503,-0.043353,-0.023961,-0.044582,-0.033704
81293,-0.005333,-0.084559,0.012102,0.034559,-0.044233,0.004474,-0.038457,-0.022581,-0.049232,-0.016955
81294,0.547198,0.404916,-0.158610,0.200423,0.078439,0.038694,0.044679,-0.027446,0.008973,0.044799


向量2：description

In [98]:
# step1：缺失值填充

jd_data['description'].fillna('', inplace=True)

/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_38964/745361120.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jd_data['description'].fillna('', inplace=True)


In [99]:
# step2：将desciption列通过jieba分词处理成列表

col_name = 'description'

def col_jieba_fun(series):
    col = series[col_name]

    # 字符串变列表
    if col.startswith("[") and col.endswith("]"):
        col = json.loads(col)
    else:
        col = re.split(",|，|/| ", col)

    # 对于中文，进入jieba前不需要添加空格
    # 不过，如果是中英文混合，就必须空格了
    col_str = " ".join(col)
    col_list = jieba.lcut(col_str, cut_all=False)
    return col_list

jd_data['description_jieba'] = jd_data.apply(col_jieba_fun, axis=1)
jd_data


/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_38964/4224979954.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jd_data['description_jieba'] = jd_data.apply(col_jieba_fun, axis=1)


,jd_id,title,category_name,tags,description,requirement,title_category,title_category_jieba,tags_jieba,title_category_tags_jieba,title_category_tags_jieba_filter,description_jieba
0,2c9207157bcbb76b017bcde2b46d04cd,市场营销负责人,市场营销,"[""营销"",""管理"",""销售"",""战略规划"",""管理工作""]",1、岗位职责:2、主导制定公司营销战略规划并参与公司整体战略规划制定。3、全面管理公司的市场...,1、电商2、线下营销,市场营销负责人 市场营销,"[市场营销, 负责人, , 市场营销]","[营销, , 管理, , 销售, , 战略规划, , 管理工作]","[市场营销, 负责人, , 市场营销, 营销, , 管理, , 销售, , 战略规划...",市场营销 负责人 市场营销 营销 管理 销售 战略规划 管理工作,"[1, 、, 岗位职责, :, 2, 、, 主导, 制定, 公司, 营销, 战略规划, 并,..."
1,2c9207157bcbb76b017bf7f2bbbb3153,会员管理经理,用户运营,"[""会员运营"",""会员管理"",""管理"",""营销"",""社群运营""]",1、岗位职责：2、1. 负责“九木杂物社”品牌全渠道会员体系的管理，规划品牌会员运营方向，包...,1、有会员管理、会员活动经验,会员管理经理 用户运营,"[会员, 管理, 经理, , 用户, 运营]","[会员, 运营, , 会员, 管理, , 管理, , 营销, , 社群, 运营]","[会员, 管理, 经理, , 用户, 运营, 会员, 运营, , 会员, 管理, , ...",会员 管理 经理 用户 运营 会员 运营 会员 管理 管理 营销 社群 运营,"[1, 、, 岗位职责, ：, 2, 、, 1, ., , 负责, “, 九木, 杂物, ..."
2,2c9207157bf94d99017c108f769405e0,产品市场经理,产品经理,"[""EXCEL"",""OFFICE"",""PPT"",""UI""]",1、•Support regional strategic planning and mar...,1、find jd,产品市场经理 产品经理,"[产品, 市场, 经理, , 产品, 经理]","[EXCEL, , OFFICE, , PPT, , UI]","[产品, 市场, 经理, , 产品, 经理, EXCEL, , OFFICE, , P...",产品 市场 经理 产品 经理 excel office ppt ui,"[1, 、, •, Support, , regional, , strategic, ..."
3,2c9207157d2f0ec5017d5a9265ba33d9,海外区域运营经理/高级经理,运营,"[""管理"",""运营"",""团队管理"",""电商"",""游戏""]",1、负责短视频业务的运营策略制定，配合完成公司的营收业务指标；2、主导团队的业务方向，包括但...,1、本科及以上学历优先，3年以上社交、直播、游戏、电商或相关行业运营工作经验；2、英语听说读...,海外区域运营经理/高级经理 运营,"[海外, 区域, 运营, 经理, , 高级, 经理, , 运营]","[管理, , 运营, , 团队, 管理, , 电商, , 游戏]","[海外, 区域, 运营, 经理, , 高级, 经理, , 运营, 管理, , 运营, ...",海外 区域 运营 经理 高级 经理 运营 管理 运营 团队 管理 电商 游戏,"[1, 、, 负责, 短, 视频, 业务, 的, 运营, 策略, 制定, , 配合, 完成..."
4,2c9207157d631abb017d6eed1c79139f,高级项目经理,高级管理,"[""项目管理"",""推广"",""归档"",""风险评估"",""交货""]",1、负责从产品建立到第一批生产交付的全过程项目管理;2、 负责实现产品要求、进度、成本、质量...,1、 本科及以上学历，计算机科学或相关专业。能用英语交流者优先;2、5年以上可穿戴相关产品项...,高级项目经理 高级管理,"[高级, 项目经理, , 高级, 管理]","[项目管理, , 推广, , 归档, , 风险, 评估, , 交货]","[高级, 项目经理, , 高级, 管理, 项目管理, , 推广, , 归档, , 风...",高级 项目经理 高级 管理 项目管理 推广 归档 风险 评估 交货,"[1, 、, 负责, 从, 产品, 建立, 到, 第一批, 生产, 交付, 的, 全过程, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
81291,8a69f6f8823a93b901823d891b761d51,技术经理,机械设备工程师,"[""材料"",""考核"",""维修"",""技术标准"",""不锈钢""]",职责描述：1、全面负责公司技术标准制、各岗位的工艺制定跟改善；2、重要设备参数收集管控包各棍...,任职要求：1、具备压延400系不锈钢材料技术相关工作经验，能对水扎四六棍扎异常原因分析解决、...,技术经理 机械设备工程师,"[技术, 经理, , 机械设备, 工程师]","[材料, , 考核, , 维修, , 技术标准, , 不锈钢]","[技术, 经理, , 机械设备, 工程师, 材料, , 考核, , 维修, , 技术...",技术 经理 机械设备 工程师 材料 考核 维修 技术标准 不锈钢,"[职责, 描述, ：, 1, 、, 全面, 负责, 公司, 技术标准, 制, 、, 各, 岗..."
81292,8a69c468826ee9b40182831bedfc5e79,中央市场总监,市场总监,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场总监,"[中央, 市场总监, , 市场总监]","[市场推广, , 医疗器械, , 语言, , 材料, , 营销]","[中央, 市场总监, , 市场总监, 市场推广, , 医疗器械, , 语言, , 材...",中央 市场总监 市场总监 市场推广 医疗器械 语言 材料 营销,"[1, ., , 把握, 行业, 市场, 发展趋势, , 分析, 全球, 市场动态, ..."
81293,8a69f6f8823a93b9018247c30aca4e3d,中央市场总监,市场/营销,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场/营销,"[中央, 市场总监, , 市场, , 营销]","[市场推广, , 医疗器械, , 语言, , 材料, , 营销]","[中央, 市场总监, , 市场, , 营销, 市场推广, , 医疗器械, , 语言,...",中央 市场总监 市场 营销 市场推广 医疗器械 语言 材料 营销,"[1, ., , 把握, 行业, 市场, 发展趋势, , 分析, 全球, 市场动态, ..."
81294,2c92071580dd3ab70180f18714e119c0,java开发工程师（不限方向）,Java,"[""java"",""微服务/性能优化"",""SQL"",""架构"",""需求分析""]",1、负责公司OA产品的功能规划、需求分析设计、技术实现和用户体验；2、负责所属模块的代码开发...,1、本科及以上学历，2年以上的Java开发及应用经验，计算机或者相关专业;2、良好的问题理解...,java开发工程师（不限方向） Java,"[java, 开发, 工程师, （, 不, 限, 方向, ）, , Java]","[java, , 微, 服务, /, 性能, 优化, , SQL, , 架构, , ...","[java, 开发, 工程师, （, 不, 限, 方向, ）, , Java, java,...",java 开发 工程师 方向 java java 服务 性能 优化 sql 架构 需求 分析,"[1, 、, 负责, 公司, OA, 产品, 的, 功能, 规划, 、, 需求, 分析, 设..."


In [101]:
# step3：过滤description_jieba，得到以空格分割的字符串

col_name = 'description_jieba'

def col_jieba_filter_fun(series):

    col_list_filter = []
    # 得到tag列表
    col_list = series[col_name]
    # print(tags_list[0])

    pun_masks_english = [",", ".", "/", "[", "]", "{", "}", "(", ")", ":", "*", "#", "!", " ", "\"", "\\"]
    pun_masks_chinese = ["，", "。", "、", "（", "）", "：", "！", "”", "“"]
    pun_masks = pun_masks_english + pun_masks_chinese

    # 过滤
    for tag in col_list:
        # 转中文简体
        tag = convert(tag, "zh-hans")
        # 转英文小写
        tag = tag.lower()

        # 过滤数字
        if tag.isdigit():
            continue
        
        # 过滤单个字符
        if len(tag) <= 1:
            continue
        
        # 过滤标点
        flag = 1
        for pun in pun_masks:
            if pun in tag:
                flag = 0
                break
        if flag == 1:
            col_list_filter.append(tag)
    return " ".join(col_list_filter)

jd_data['description_jieba_filter'] = jd_data.apply(col_jieba_filter_fun, axis=1)
jd_data

/var/folders/99/wft1658s2mv39cv4wj_rcpqh0000gp/T/ipykernel_38964/2906327983.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jd_data['description_jieba_filter'] = jd_data.apply(col_jieba_filter_fun, axis=1)


,jd_id,title,category_name,tags,description,requirement,title_category,title_category_jieba,tags_jieba,title_category_tags_jieba,title_category_tags_jieba_filter,description_jieba,description_jieba_filter
0,2c9207157bcbb76b017bcde2b46d04cd,市场营销负责人,市场营销,"[""营销"",""管理"",""销售"",""战略规划"",""管理工作""]",1、岗位职责:2、主导制定公司营销战略规划并参与公司整体战略规划制定。3、全面管理公司的市场...,1、电商2、线下营销,市场营销负责人 市场营销,"[市场营销, 负责人, , 市场营销]","[营销, , 管理, , 销售, , 战略规划, , 管理工作]","[市场营销, 负责人, , 市场营销, 营销, , 管理, , 销售, , 战略规划...",市场营销 负责人 市场营销 营销 管理 销售 战略规划 管理工作,"[1, 、, 岗位职责, :, 2, 、, 主导, 制定, 公司, 营销, 战略规划, 并,...",岗位职责 主导 制定 公司 营销 战略规划 参与 公司 整体 战略规划 制定 全面 管理 公...
1,2c9207157bcbb76b017bf7f2bbbb3153,会员管理经理,用户运营,"[""会员运营"",""会员管理"",""管理"",""营销"",""社群运营""]",1、岗位职责：2、1. 负责“九木杂物社”品牌全渠道会员体系的管理，规划品牌会员运营方向，包...,1、有会员管理、会员活动经验,会员管理经理 用户运营,"[会员, 管理, 经理, , 用户, 运营]","[会员, 运营, , 会员, 管理, , 管理, , 营销, , 社群, 运营]","[会员, 管理, 经理, , 用户, 运营, 会员, 运营, , 会员, 管理, , ...",会员 管理 经理 用户 运营 会员 运营 会员 管理 管理 营销 社群 运营,"[1, 、, 岗位职责, ：, 2, 、, 1, ., , 负责, “, 九木, 杂物, ...",岗位职责 负责 九木 杂物 品牌 渠道 会员 体系 管理 规划 品牌 会员 运营 方向 包括...
2,2c9207157bf94d99017c108f769405e0,产品市场经理,产品经理,"[""EXCEL"",""OFFICE"",""PPT"",""UI""]",1、•Support regional strategic planning and mar...,1、find jd,产品市场经理 产品经理,"[产品, 市场, 经理, , 产品, 经理]","[EXCEL, , OFFICE, , PPT, , UI]","[产品, 市场, 经理, , 产品, 经理, EXCEL, , OFFICE, , P...",产品 市场 经理 产品 经理 excel office ppt ui,"[1, 、, •, Support, , regional, , strategic, ...",support regional strategic planning and market...
3,2c9207157d2f0ec5017d5a9265ba33d9,海外区域运营经理/高级经理,运营,"[""管理"",""运营"",""团队管理"",""电商"",""游戏""]",1、负责短视频业务的运营策略制定，配合完成公司的营收业务指标；2、主导团队的业务方向，包括但...,1、本科及以上学历优先，3年以上社交、直播、游戏、电商或相关行业运营工作经验；2、英语听说读...,海外区域运营经理/高级经理 运营,"[海外, 区域, 运营, 经理, , 高级, 经理, , 运营]","[管理, , 运营, , 团队, 管理, , 电商, , 游戏]","[海外, 区域, 运营, 经理, , 高级, 经理, , 运营, 管理, , 运营, ...",海外 区域 运营 经理 高级 经理 运营 管理 运营 团队 管理 电商 游戏,"[1, 、, 负责, 短, 视频, 业务, 的, 运营, 策略, 制定, , 配合, 完成...",负责 视频 业务 运营 策略 制定 配合 完成 公司 营收 业务 指标 主导 团队 业务 方...
4,2c9207157d631abb017d6eed1c79139f,高级项目经理,高级管理,"[""项目管理"",""推广"",""归档"",""风险评估"",""交货""]",1、负责从产品建立到第一批生产交付的全过程项目管理;2、 负责实现产品要求、进度、成本、质量...,1、 本科及以上学历，计算机科学或相关专业。能用英语交流者优先;2、5年以上可穿戴相关产品项...,高级项目经理 高级管理,"[高级, 项目经理, , 高级, 管理]","[项目管理, , 推广, , 归档, , 风险, 评估, , 交货]","[高级, 项目经理, , 高级, 管理, 项目管理, , 推广, , 归档, , 风...",高级 项目经理 高级 管理 项目管理 推广 归档 风险 评估 交货,"[1, 、, 负责, 从, 产品, 建立, 到, 第一批, 生产, 交付, 的, 全过程, ...",负责 产品 建立 第一批 生产 交付 全过程 项目管理 负责 实现 产品 要求 进度 成本 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
81291,8a69f6f8823a93b901823d891b761d51,技术经理,机械设备工程师,"[""材料"",""考核"",""维修"",""技术标准"",""不锈钢""]",职责描述：1、全面负责公司技术标准制、各岗位的工艺制定跟改善；2、重要设备参数收集管控包各棍...,任职要求：1、具备压延400系不锈钢材料技术相关工作经验，能对水扎四六棍扎异常原因分析解决、...,技术经理 机械设备工程师,"[技术, 经理, , 机械设备, 工程师]","[材料, , 考核, , 维修, , 技术标准, , 不锈钢]","[技术, 经理, , 机械设备, 工程师, 材料, , 考核, , 维修, , 技术...",技术 经理 机械设备 工程师 材料 考核 维修 技术标准 不锈钢,"[职责, 描述, ：, 1, 、, 全面, 负责, 公司, 技术标准, 制, 、, 各, 岗...",职责 描述 全面 负责 公司 技术标准 岗位 工艺 制定 改善 重要 设备 参数 收集 管控...
81292,8a69c468826ee9b40182831bedfc5e79,中央市场总监,市场总监,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场总监,"[中央, 市场总监, , 市场总监]","[市场推广, , 医疗器械, , 语言, , 材料, , 营销]","[中央, 市场总监, , 市场总监, 市场推广, , 医疗器械, , 语言, , 材...",中央 市场总监 市场总监 市场推广 医疗器械 语言 材料 营销,"[1, ., , 把握, 行业, 市场, 发展趋势, , 分析, 全球, 市场动态, ...",把握 行业 市场 发展趋势 分析 全球 市场动态 拟定 公司 产品 市场 发展 战略规划 结...
81293,8a69f6f8823a93b9018247c30aca4e3d,中央市场总监,市场/营销,"[""市场推广"",""医疗器械"",""语言"",""材料"",""营销""]",1. 把握行业市场发展趋势，分析全球市场动态，拟定公司产品市场发展战略规划；2. 结合公司战...,1. 本科及以上学历，医学、医疗器械相关专业者优先；2. 10年及以上市场工作经验，5年以上...,中央市场总监 市场/营销,"[中央, 市场总监, , 市场, , 营销]","[市场推广, , 医疗器械, , 语言, , 材料, , 营销]","[中央, 市场总监, , 市场, , 营销, 市场推广, , 医疗器械, , 语言,...",中央 市场总监 市场 营销 市场推广 医疗器械 语言 材料 营销,"[1, ., , 把握, 行业, 市场, 发展趋势, , 分析, 全球, 市场动态, ...",把握 行业 市场 发展趋势 分析 全球 市场动态 拟定 公司 产品 市场 发展 战略规划 结...
81294,2c92071580dd3ab70180f18714e119c0,java开发工程师（不限方向）,Java,"[""java"",""微服务/性能优化"",""SQL"",""架构"",""需求分析""]",1、负责公司OA产品的功能规划、需求分析设计、技术实现和用户体验；2、负责所属模块的代码开发...,1、本科及以上学历，2年以上的Java开发及应用经验，计算机或者相关专业;2、良好的问题理解...,java开发工程师（不限方向） Java,"[java, 开发, 工程师, （, 不, 限

,description_jieba,description_jieba_filter
0,"[1, 、, 岗位职责, :, 2, 、, 主导, 制定, 公司, 营销, 战略规划, 并,...",岗位职责 主导 制定 公司 营销 战略规划 参与 公司 整体 战略规划 制定 全面 管理 公...
1,"[1, 、, 岗位职责, ：, 2, 、, 1, ., , 负责, “, 九木, 杂物, ...",岗位职责 负责 九木 杂物 品牌 渠道 会员 体系 管理 规划 品牌 会员 运营 方向 包括...
2,"[1, 、, •, Support, , regional, , strategic, ...",support regional strategic planning and market...
3,"[1, 、, 负责, 短, 视频, 业务, 的, 运营, 策略, 制定, , 配合, 完成...",负责 视频 业务 运营 策略 制定 配合 完成 公司 营收 业务 指标 主导 团队 业务 方...
4,"[1, 、, 负责, 从, 产品, 建立, 到, 第一批, 生产, 交付, 的, 全过程, ...",负责 产品 建立 第一批 生产 交付 全过程 项目管理 负责 实现 产品 要求 进度 成本 ...
...,...,...
81291,"[职责, 描述, ：, 1, 、, 全面, 负责, 公司, 技术标准, 制, 、, 各, 岗...",职责 描述 全面 负责 公司 技术标准 岗位 工艺 制定 改善 重要 设备 参数 收集 管控...
81292,"[1, ., , 把握, 行业, 市场, 发展趋势, , 分析, 全球, 市场动态, ...",把握 行业 市场 发展趋势 分析 全球 市场动态 拟定 公司 产品 市场 发展 战略规划 结...
81293,"[1, ., , 把握, 行业, 市场, 发展趋势, , 分析, 全球, 市场动态, ...",把握 行业 市场 发展趋势 分析 全球 市场动态 拟定 公司 产品 市场 发展 战略规划 结...
81294,"[1, 、, 负责, 公司, OA, 产品, 的, 功能, 规划, 、, 需求, 分析, 设...",负责 公司 oa 产品 功能 规划 需求 分析 设计 技术 实现 用户 体验 负责 所属 模...


In [105]:
# step4：将description_jieba_filter转成tfidf向量

def get_tfidf(df, col_name):
    # col_name必须是以空格分割的字符串
    text = df[col_name]
    
    vectorizer = TfidfVectorizer()
    # 返回的是nparray
    vector = vectorizer.fit_transform(text)
    return pd.DataFrame(vector.toarray()), vectorizer

tfidf, vectorizer = get_tfidf(jd_data, 'description_jieba_filter')
tfidf

,0,1,2,3,4,5,6,7,8,9,...,16274,16275,16276,16277,16278,16279,16280,16281,16282,16283
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81291,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81292,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81293,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [106]:
vectorizer.vocabulary_

{'岗位职责': 9223,
 '主导': 5358,
 '制定': 6945,
 '公司': 6447,
 '营销': 14231,
 '战略规划': 10100,
 '参与': 7441,
 '整体': 10954,
 '全面': 6424,
 '管理': 13346,
 '市场策划': 9359,
 '品牌': 7959,
 '建设': 9637,
 '工作': 9247,
 '销售': 15689,
 '渠道': 12163,
 '开发': 9654,
 '管理工作': 13357,
 '以及': 5806,
 '电子商务': 12627,
 '发展': 7566,
 '创建': 6872,
 '培养': 8289,
 '一流': 4899,
 '市场营销': 9361,
 '团队': 8126,
 '高效': 16206,
 '组织': 13568,
 '规划': 14420,
 '落实': 14239,
 '积极': 13104,
 '完成': 8819,
 '目标': 12812,
 '业绩': 5179,
 '利润': 6910,
 '达成': 15220,
 '实现': 8896,
 '企业': 5863,
 '经营': 13631,
 '负责': 14805,
 '完善': 8816,
 '各项': 7756,
 '政策': 10841,
 '规范': 14436,
 '制度': 6947,
 '不断': 5064,
 '适应': 15378,
 '需要': 15923,
 '协同': 7280,
 '合作伙伴': 7764,
 '紧密': 13475,
 '合作': 7763,
 '关系': 6518,
 '经销商': 13642,
 '代理商': 5784,
 '良好': 14123,
 '保持': 6113,
 '稳健': 13158,
 '快速': 9893,
 '持续': 10465,
 '任职': 5859,
 '资格': 14951,
 '大学本科': 8576,
 '以上学历': 5796,
 '年龄': 9489,
 '以下': 5798,
 '以上': 5795,
 '高管': 16225,
 '经验': 13643,
 '最近': 11309,
 '担任': 10405,
 '过年': 15233,
 '营业额': 1422

In [107]:
# step5：将tfidf降维到n维

def get_tfidf_pca(tfidf, n=20):
    pca = PCA(n_components=n)
    tfidf_pca = pca.fit_transform(tfidf)
    tfidf_pca = pd.DataFrame(tfidf_pca)
    return tfidf_pca

tfidf_pca = get_tfidf_pca(tfidf, 10)
tfidf_pca

,0,1,2,3,4,5,6,7,8,9
0,-0.203607,0.048253,0.060794,0.056752,0.007372,-0.028802,0.052828,-0.005516,-0.057960,-0.137604
1,-0.092220,0.055354,-0.116769,-0.037503,0.006249,0.062780,0.132458,0.006080,-0.084890,-0.016954
2,-0.073116,-0.181972,-0.229605,-0.014929,-0.141456,-0.253847,0.005384,-0.038444,-0.011032,0.194607
3,-0.094117,0.098947,-0.037174,-0.077933,0.025272,0.065636,0.080129,0.017128,-0.095360,0.007027
4,0.039447,-0.033039,0.064930,0.077024,-0.038287,0.088010,0.078232,-0.163983,0.050215,-0.011412
...,...,...,...,...,...,...,...,...,...,...
81291,-0.061453,-0.086786,-0.043291,0.013254,-0.048909,0.012192,0.025798,-0.028231,0.064829,-0.021774
81292,-0.119764,0.200981,0.100039,0.127447,0.020233,-0.043395,-0.001541,0.025444,-0.053329,-0.110610
81293,-0.119764,0.200981,0.100039,0.127447,0.020233,-0.043395,-0.001541,0.025444,-0.053329,-0.110610
81294,0.407429,-0.056525,0.382055,0.189822,0.193643,-0.226503,0.220974,0.407579,0.168398,0.101317
